In [1]:
# Librerías generales

import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

### Lectura de datos

In [2]:
data_folder_name = 'dataSelected'

def getFileNames(folder_name):
    
    path = os.path.join(os.getcwd(),'..','..',folder_name)
    file_names = os.listdir(path)

    return file_names 

print(getFileNames(data_folder_name))

['x_sub_train_aug.npy', 'x_sub_val.npy', 'y_sub_train_aug_0.npy', 'y_sub_train_aug_1.npy', 'y_sub_val_0.npy', 'y_sub_val_1.npy']


In [3]:
file_names = getFileNames(data_folder_name)

# Lectura de archivos '.npy' 

x_test = np.load('../'+'../'+data_folder_name+'/'+'x_sub_val.npy')
x_train = np.load('../'+'../'+data_folder_name+'/'+'x_sub_train_aug.npy')

y_train_coarse = np.load('../'+'../'+data_folder_name+'/'+'y_sub_train_aug_1.npy')
y_train_fine = np.load('../'+'../'+data_folder_name+'/'+'y_sub_train_aug_0.npy')

y_test_coarse = np.load('../'+'../'+data_folder_name+'/'+'y_sub_val_1.npy')
y_test_fine = np.load('../'+'../'+data_folder_name+'/'+'y_sub_val_0.npy')

# Lectura de archivos '.pck'

with open('../'+'../'+'dl-itba-cifar-100-2024-q-1'+'/'+'coarse_label_names.pck', "rb") as f:
    coarse_label_names = pickle.load(f)
f.close()

with open('../'+'../'+'dl-itba-cifar-100-2024-q-1'+'/'+'fine_label_names.pck', "rb") as f:
    fine_label_names = pickle.load(f)
f.close()

print('Dimensiones de los datos:')
print(x_test.shape)
print(x_train.shape)
print(y_train_coarse.shape)
print(y_train_fine.shape)

print('Cantidad de clases:')
print(np.shape(coarse_label_names))
print(np.shape(fine_label_names))

Dimensiones de los datos:
(10000, 32, 32, 3)
(1040000, 32, 32, 3)
(1040000,)
(1040000,)
Cantidad de clases:
(20,)
(100,)


### Armado del modelo

In [4]:
from keras.layers import Input, Dense, Dropout, Flatten, LayerNormalization
from keras.models import Model
from keras.regularizers import l2

In [5]:
# Input layer
input_layer = Input(shape=(32,32,3), name='matrix_input')

#Normalization layer
normalization_layer = LayerNormalization(name='normalization_layer')(input_layer)

# Flatten layer
flatten_layer = Flatten(name='flatten_layer')(normalization_layer)

# Dense layer 0 - 64 units
dense_0_layer = Dense(
    units                   = 64,
    activation              = 'sigmoid',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'dense_0_layer'
)(flatten_layer)

#Dense layer 1 - 128 units
dense_1_layer = Dense(
    units                   = 128,
    activation              = 'tanh',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'dense_1_layer'
)(dense_0_layer)

#Droput layer 1
dropout_1_layer = Dropout(
    rate                    = 0.2,
    name                    = 'dropout_1_layer'
)(dense_1_layer)

#Dense layer 2 - 256 units
dense_2_layer = Dense(
    units                   = 256,
    activation              = 'relu',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = l2(0.01),
    bias_regularizer        = l2(0.01),
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'dense_2_layer'
)(dropout_1_layer)

#Droput layer 2
dropout_2_layer = Dropout(
    rate                    = 0.2,
    name                    = 'dropout_2_layer'
)(dense_2_layer)

#Dense layer 3 - 128 units
dense_3_layer = Dense(
    units                   = 128,
    activation              = 'relu',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'dense_3_layer'
)(dropout_2_layer)

#Dense layer 4 - 64 units
dense_4_layer = Dense(
    units                   = 64,
    activation              = 'relu',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'dense_4_layer'
)(dense_3_layer)

#Output layer 0 - 100 units
fine_output = Dense(
    units                   = 100,
    activation              = 'softmax',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'fine_output'
)(dense_4_layer)

"""#Output layer 1 - 20 units
coarse_output = Dense(
    units                   = 20,
    activation              = 'softmax',
    kernel_initializer      = 'glorot_uniform',
    bias_initializer        = 'glorot_uniform',
    kernel_regularizer      = None,
    bias_regularizer        = None,
    activity_regularizer    = None,
    kernel_constraint       = None,
    bias_constraint         = None,
    lora_rank               = None,
    name                    = 'coarse_output'
)(dense_4_layer)"""

# Defino el modelo con dos salidas
myModel = Model(
    inputs                  = input_layer,
    #outputs                 = [fine_output, coarse_output]
    outputs                 = [fine_output]
)

# Print model summary
myModel.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ matrix_input (InputLayer)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_layer             │ (None, 32, 32, 3)      │             6 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer (Flatten)         │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_0_layer (Dense)           │ (None, 64)             │       196,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1_layer (Dense)           │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1_layer (Dropout)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2_layer (Dense)           │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2_layer (Dropout)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3_layer (Dense)           │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4_layer (Dense)           │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 285,674 (1.09 MB)

 Trainable params: 285,674 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

### Compilación del modelo

In [6]:
from keras.optimizers import Adam

In [7]:
myModel.compile(
    optimizer               = Adam(learning_rate=1e-3),
    #loss                    = {'fine_output': 'sparse_categorical_crossentropy', 'coarse_output': 'sparse_categorical_crossentropy'},
    loss                    = {'fine_output': 'sparse_categorical_crossentropy'},
    loss_weights            = None,
    #metrics                 = {'fine_output': 'categorical_accuracy', 'coarse_output': 'categorical_accuracy'},
    metrics                 = {'fine_output': 'categorical_accuracy'},
    weighted_metrics        = None,
    run_eagerly             = False,
    steps_per_execution     = 1,
    jit_compile             = "auto",
    auto_scale_loss         = True,
)

### Fit del modelo

Primero se van a definir algunos callbacks.

In [8]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard, LambdaCallback

In [9]:
#AGREGUE ACA UN PAR QUE ESTABAN EN EL OTRO
rlrop = ReduceLROnPlateau(
    monitor                 = "val_accuracy",
    factor                  = 0.2,
    patience                = 5,
    verbose                 = 1,
    min_lr                  = 1e-6
)

es = EarlyStopping(
    monitor                 = "val_loss",
    patience                = 10,
    verbose                 = 1,
    restore_best_weights    = True
)

mc = ModelCheckpoint(
    "best_weights.weights.h5",
    monitor                 = "val_loss",
    verbose                 = 1,
    save_best_only          = True,
    save_weights_only       = True,
)

tb = TensorBoard(
    log_dir                 = "logs"
)

In [10]:
# Shuffle de los datos
indexes_train   = np.arange(len(x_train))
indexes_test    = np.arange(len(x_test))

np.random.shuffle(indexes_train)
np.random.shuffle(indexes_test)

# Shuffle de datos de train

x_train_dup         = x_train[indexes_train]
y_train_coarse_dup  = y_train_coarse[indexes_train]
y_train_fine_dup    = y_train_fine[indexes_train]

# Shuffle de datos de test
x_test_dup         = x_test[indexes_test]
y_test_coarse_dup  = y_test_coarse[indexes_test]
y_test_fine_dup    = y_test_fine[indexes_test]

In [21]:
from keras.utils import to_categorical

# Convertir etiquetas a formato one-hot
y_train_fine_dup = to_categorical(y_train_fine_dup, num_classes=100)
y_train_coarse_dup = to_categorical(y_train_coarse_dup, num_classes=20)

y_test_fine_dup = to_categorical(y_test_fine_dup, num_classes=100)
y_test_coarse_dup = to_categorical(y_test_coarse_dup, num_classes=20)

In [11]:
history = myModel.fit(
    x                       = x_train_dup,
    #y                       = {'fine_output': y_train_fine_dup, 'coarse_output': y_train_coarse_dup},
    y                       = {'fine_output': y_train_fine_dup},
    batch_size              = 64,
    epochs                  = 5,
    verbose                 = "auto",
    callbacks               = [LambdaCallback(on_epoch_end=lambda epoch, logs: print(logs if logs is not None else "No logs available"))],
    validation_split        = 0.0,
    #validation_data         = (x_test_dup, {'fine_output': y_test_fine_dup, 'coarse_output': y_test_coarse_dup}),
    validation_data         = (x_test_dup, {'fine_output': y_test_fine_dup}),
    shuffle                 = True,
    class_weight            = None,
    sample_weight           = None,
    initial_epoch           = 0,
    steps_per_epoch         = None,
    validation_steps        = None,
    validation_batch_size   = None,
    validation_freq         = 1,
)


Epoch 1/5
16249/16250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - categorical_accuracy: 0.0098 - loss: 4.3161{'categorical_accuracy': 0.01022980734705925, 'loss': 4.227025032043457, 'val_categorical_accuracy': 0.013299999758601189, 'val_loss': 4.117843151092529}
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 116s 7ms/step - categorical_accuracy: 0.0098 - loss: 4.3161 - val_categorical_accuracy: 0.0133 - val_loss: 4.1178
Epoch 2/5
16248/16250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - categorical_accuracy: 0.0101 - loss: 4.1572{'categorical_accuracy': 0.010146154090762138, 'loss': 4.145777225494385, 'val_categorical_accuracy': 0.012299999594688416, 'val_loss': 4.066841125488281}
16250/16250 ━━━━━━━━━━━━━━━━━━━━ 112s 7ms/step - categorical_accuracy: 0.0101 - loss: 4.1572 - val_categorical_accuracy: 0.0123 - val_loss: 4.0668
Epoch 3/5
16249/16250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - categorical_accuracy: 0.0098 - loss: 4.1238{'categorical_accuracy': 0.00981346145272255, 'loss': 4.120007514953613, 'val_categorical_accurac